In [55]:
import numpy as np
import pandas as pd
import tensorflow as tf
from math import sqrt
import holidays
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import mean_absolute_error
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense
from deepctr.models import WDL
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train_df = pd.read_csv('./dataset/train.csv', sep=',', nrows=100000)

In [3]:
train_df.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


#### renaming the columns hotel_cluster and is_booking to hotel_id and rating

In [4]:
train_df = train_df.rename(columns={'hotel_cluster': 'hotel_id', 'is_booking': 'rating'})

In [5]:
train_df = train_df.drop(['orig_destination_distance'],axis=1)
train_df = train_df.dropna()

In [6]:
train_df.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,user_id,is_mobile,is_package,channel,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,rating,cnt,hotel_continent,hotel_country,hotel_market,hotel_id
0,2014-08-11 07:46:59,2,3,66,348,48862,12,0,1,9,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,12,0,1,9,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,12,0,0,9,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,93,0,0,3,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,93,0,0,3,...,0,1,14984,1,0,1,2,50,1457,21


#### we are listing out a new columns for america and europe to see if customers are checking in holidays

In [11]:
# Initially, we define hoilday dates for america
holidays_america = holidays.UnitedStates()
holidays_canada = holidays.Canada()

#### Setting up new columns in the train_df dataframe. 1 indicates that customers have checked on holidays and 0 indicate that customers have not checked on holidays

In [12]:
train_df['america_checkin'] = train_df['srch_ci'].apply(lambda holiday : 1 if holiday in (holidays_canada or holidays_america) else 0)
train_df['america_checkout'] = train_df['srch_co'].apply(lambda holiday : 1 if holiday in (holidays_canada or holidays_america) else 0)

In [14]:
# Now, we define hoilday dates for europe
holidays_uk = holidays.UnitedKingdom()
holidays_germany = holidays.Germany()

In [15]:
train_df['europe_checkin'] = train_df['srch_ci'].apply(lambda holiday : 1 if holiday in (holidays_uk or holidays_germany) else 0)
train_df['europe_checkout'] = train_df['srch_co'].apply(lambda holiday : 1 if holiday in (holidays_uk or holidays_germany) else 0)

In [16]:
train_df.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,user_id,is_mobile,is_package,channel,...,rating,cnt,hotel_continent,hotel_country,hotel_market,hotel_id,america_checkin,america_checkout,europe_checkin,europe_checkout
0,2014-08-11 07:46:59,2,3,66,348,48862,12,0,1,9,...,0,3,2,50,628,1,0,0,0,0
1,2014-08-11 08:22:12,2,3,66,348,48862,12,0,1,9,...,1,1,2,50,628,1,0,0,0,0
2,2014-08-11 08:24:33,2,3,66,348,48862,12,0,0,9,...,0,1,2,50,628,1,0,0,0,0
3,2014-08-09 18:05:16,2,3,66,442,35390,93,0,0,3,...,0,1,2,50,1457,80,0,0,0,0
4,2014-08-09 18:08:18,2,3,66,442,35390,93,0,0,3,...,0,1,2,50,1457,21,0,0,0,0


#### As we do not get meaningful insights from datetime fields, we extract month from it and drop them

In [17]:
train_df['month_ci'] = pd.DatetimeIndex(train_df['srch_ci'])
train_df['month_co'] = pd.DatetimeIndex(train_df['srch_co'])
train_df['month_click'] = pd.DatetimeIndex(train_df['date_time'])

In [18]:
train_df = train_df.drop(['srch_ci', 'srch_co', 'date_time'], axis = 1)

In [19]:
train_df.shape

(99929, 27)

### Feature engineering is done where the inputs (both categorical and continuous) are fed to wide and deep neural networks

In [24]:
# All categorical attributes are put in sparse features
categotical_cols = []
for col in train_df.columns:
    if col not in ['hotel_market', 'rating']:
        categotical_cols.append(col)
sparse_features = categotical_cols

In [28]:
dense_features = ['hotel_market']
target = ['rating']

#### In the preprocessing step, we encode the sparse categorical features for embedding using Label Encoding

In [29]:
for feature in sparse_features:
    label = LabelEncoder()
    train_df[feature] = label.fit_transform(train_df[feature])

#### We use normalization like MinMaxScaler methods for dense featues as they are discretized to buckets

In [30]:
mms = MinMaxScaler(feature_range=(0,1))
train_df[dense_features] = mms.fit_transform(train_df[dense_features])

#### We use embedding techniques to turn sparse features into dense vectors. We concatenate dense features to the fully connected layer's input tensors.

In [31]:
fixlen_feature_columns = [SparseFeat(feat, train_df[feat].nunique(), embedding_dim = 4)
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [33]:
train_data, test_data = train_test_split(train_df, test_size = 0.3)

In [34]:
train_data_input = {name : train_data[name].values for name in feature_names}
test_data_input = {name : test_data[name].values for name in feature_names}

#### Now, we apply tensorflow's WDL library and compile the model

In [64]:
model = WDL(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(128, 128), l2_reg_embedding=1e-5,
            l2_reg_dnn = 0, l2_reg_linear = 1e-5, seed = 1024, dnn_activation = 'relu', task = 'binary')


model.compile("adam", "mse", metrics=['mse'])

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'


In [65]:
history = model.fit(train_data_input, train_data[target].values, batch_size = 256, epochs = 10, verbose = 2, validation_split = 0.2)

Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 29s - loss: 0.0844 - mean_squared_error: 0.0842 - val_loss: 0.0674 - val_mean_squared_error: 0.0671
Epoch 2/10
55960/55960 - 20s - loss: 0.0624 - mean_squared_error: 0.0617 - val_loss: 0.0685 - val_mean_squared_error: 0.0674
Epoch 3/10
55960/55960 - 27s - loss: 0.0037 - mean_squared_error: 0.0025 - val_loss: 0.0691 - val_mean_squared_error: 0.0680
Epoch 4/10
55960/55960 - 20s - loss: 0.0014 - mean_squared_error: 4.1304e-04 - val_loss: 0.0703 - val_mean_squared_error: 0.0693
Epoch 5/10
55960/55960 - 22s - loss: 9.2752e-04 - mean_squared_error: 1.2836e-04 - val_loss: 0.0696 - val_mean_squared_error: 0.0689
Epoch 6/10
55960/55960 - 20s - loss: 7.5528e-04 - mean_squared_error: 1.2324e-04 - val_loss: 0.0708 - val_mean_squared_error: 0.0702
Epoch 7/10
55960/55960 - 20s - loss: 6.2070e-04 - mean_squared_error: 9.7389e-05 - val_loss: 0.0733 - val_mean_squared_error: 0.0728
Epoch 8/10
55960/55960 - 20s - loss: 5.0195e-04

In [66]:
pred = model.predict(test_data_input, batch_size=256)

In [67]:
def rmse_score(pred):
    return np.round(math.sqrt(mean_squared_error(test_data[target].values, pred)),2)

In [69]:
rmse_val = rmse_score(pred)
print("RMSE = ", rmse_val)

RMSE =  0.28


#### We perfome hyper parameter tuning using Random Search CV 

In [70]:
rand_parameters = {'dnn_hidden_units' : [(1,1),(2,2),(4,4),(8,8),(29,29),(128,128),(256,256)],
                    'l2_reg_linear':[1e-5, 1e-3, 1e-1, 1, 10],
                    'l2_reg_embedding':[1e-7, 1e-5, 1e-3, 1e-1, 1],
                     'l2_reg_dnn':[0, 0.2, 2, 4],
                      'dnn_dropout':[0, 0.2, 0.4, 0.6, 0.8, 1]} 
# rand_parameters = {'dnn_hidden_units' : [(1,1),(2,2)],
#                     'l2_reg_linear':[1e-5],
#                     'l2_reg_embedding':[1e-7],
#                      'l2_reg_dnn':[0.2],
#                       'dnn_dropout':[0.2]} 

In [71]:
dnn_hidden_units_rmse = {}
for k in rand_parameters['dnn_hidden_units']:
    model = WDL(linear_feature_columns, dnn_feature_columns, dnn_hidden_units = k, seed = 1024, task ='binary')
    model.compile("adam", "mse", metrics=['mse'])
    history = model.fit(train_data_input, train_data[target].values, batch_size=256, validation_split=0.2, epochs=10, verbose=2)
    pred = model.predict(test_data_input, batch_size=256)
    dnn_hidden_units_rmse[k]={"RMSE": np.round(math.sqrt(mean_squared_error(test_data[target].values, pred)),2)}
dnn_hidden_units_rmse

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 40s - loss: 0.1051 - mean_squared_error: 0.1051 - val_loss: 0.0746 - val_mean_squared_error: 0.0745
Epoch 2/10
55960/55960 - 29s - loss: 0.0754 - mean_squared_error: 0.0752 - val_loss: 0.0726 - val_mean_squared_error: 0.0724
Epoch 3/10
55960/55960 - 31s - loss: 0.0739 - mean_squared_error: 0.0737 - val_loss: 0.0718 - val_mean_squared_error: 0.0715
Epoch 4/10
55960/55960 - 26s - loss: 0.0727 - mean_squared_error: 0.0723 - val_loss: 0.0713 - val_mean_squared_error: 0.0708
Epoch 5/10
55960/55960 - 26s - loss: 0.0714 - mean_squared_error: 0.0709 - val_loss: 0.0708 - val_mean_squared_error: 0.0702
Epoch 6/10
55960/55960 - 27s - loss: 0.0702 - mean_squared_error: 0.0695 - val_loss: 0.0705 - val

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 40s - loss: 0.1038 - mean_squared_error: 0.1037 - val_loss: 0.0743 - val_mean_squared_error: 0.0742
Epoch 2/10
55960/55960 - 29s - loss: 0.0754 - mean_squared_error: 0.0752 - val_loss: 0.0725 - val_mean_squared_error: 0.0723
Epoch 3/10
55960/55960 - 29s - loss: 0.0739 - mean_squared_error: 0.0737 - val_loss: 0.0718 - val_mean_squared_error: 0.0715
Epoch 4/10
55960/55960 - 28s - loss: 0.0727 - mean_squared_error: 0.0723 - val_loss: 0.0712 - val_mean_squared_error: 0.0708
Epoch 5/10
55960/55960 - 28s - loss: 0.0714 - mean_squared_error: 0.0708 - val_loss: 0.0708 - val_mean_squared_error: 0.0702
Epoch 6/10
55960/55960 - 28s - loss: 0.0701 - mean_squared_error: 0.0694 - val_loss: 0.0705 - val

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 41s - loss: 0.0932 - mean_squared_error: 0.0931 - val_loss: 0.0722 - val_mean_squared_error: 0.0719
Epoch 2/10
55960/55960 - 29s - loss: 0.0726 - mean_squared_error: 0.0721 - val_loss: 0.0693 - val_mean_squared_error: 0.0685
Epoch 3/10
55960/55960 - 29s - loss: 0.0657 - mean_squared_error: 0.0645 - val_loss: 0.0697 - val_mean_squared_error: 0.0680
Epoch 4/10
55960/55960 - 29s - loss: 0.0433 - mean_squared_error: 0.0410 - val_loss: 0.0767 - val_mean_squared_error: 0.0738
Epoch 5/10
55960/55960 - 29s - loss: 0.0091 - mean_squared_error: 0.0061 - val_loss: 0.0761 - val_mean_squared_error: 0.0730
Epoch 6/10
55960/55960 - 29s - loss: 0.0046 - mean_squared_error: 0.0018 - val_loss: 0.0732 - val

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 41s - loss: 0.0947 - mean_squared_error: 0.0945 - val_loss: 0.0723 - val_mean_squared_error: 0.0720
Epoch 2/10
55960/55960 - 28s - loss: 0.0714 - mean_squared_error: 0.0708 - val_loss: 0.0683 - val_mean_squared_error: 0.0672
Epoch 3/10
55960/55960 - 28s - loss: 0.0577 - mean_squared_error: 0.0560 - val_loss: 0.0732 - val_mean_squared_error: 0.0711
Epoch 4/10
55960/55960 - 28s - loss: 0.0140 - mean_squared_error: 0.0114 - val_loss: 0.0742 - val_mean_squared_error: 0.0713
Epoch 5/10
55960/55960 - 28s - loss: 0.0038 - mean_squared_error: 0.0013 - val_loss: 0.0747 - val_mean_squared_error: 0.0724
Epoch 6/10
55960/55960 - 28s - loss: 0.0025 - mean_squared_error: 4.3650e-04 - val_loss: 0.0707 -

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'


Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 47s - loss: 0.0897 - mean_squared_error: 0.0895 - val_loss: 0.0684 - val_mean_squared_error: 0.0681
Epoch 2/10
55960/55960 - 32s - loss: 0.0653 - mean_squared_error: 0.0645 - val_loss: 0.0697 - val_mean_squared_error: 0.0685
Epoch 3/10
55960/55960 - 32s - loss: 0.0149 - mean_squared_error: 0.0133 - val_loss: 0.0699 - val_mean_squared_error: 0.0680
Epoch 4/10
55960/55960 - 31s - loss: 0.0022 - mean_squared_error: 5.2480e-04 - val_loss: 0.0697 - val_mean_squared_error: 0.0682
Epoch 5/10
55960/55960 - 32s - loss: 0.0015 - mean_squared_error: 1.5974e-04 - val_loss: 0.0705 - val_mean_squared_error: 0.0693
Epoch 6/10
55960/55960 - 32s - loss: 0.0012 - mean_squared_error: 1.0986e-04 - val_loss: 0.0699 - val_mean_squared_error: 0.0689
Epoch 7/10
55960/55960 - 32s - loss: 9.3780e-04 - mean_squared_error: 6.9171e-05 - val_loss: 0.0722 - val_mean_squared_error: 0.0714
Epoch 8/10
55960/55960 - 32s - loss: 7.8194e-04 - mean_

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 52s - loss: 0.0835 - mean_squared_error: 0.0834 - val_loss: 0.0675 - val_mean_squared_error: 0.0671
Epoch 2/10
55960/55960 - 36s - loss: 0.0602 - mean_squared_error: 0.0595 - val_loss: 0.0691 - val_mean_squared_error: 0.0680
Epoch 3/10
55960/55960 - 37s - loss: 0.0030 - mean_squared_error: 0.0019 - val_loss: 0.0717 - val_mean_squared_error: 0.0706
Epoch 4/10
55960/55960 - 37s - loss: 0.0012 - mean_squared_error: 2.2638e-04 - val_loss: 0.0733 - val_mean_squared_error: 0.0724
Epoch 5/10
55960/55960 - 36s - loss: 8.9764e-04 - mean_squared_error: 1.2303e-04 - val_loss: 0.0747 - val_mean_squared_error: 0.0740
Epoch 6/10
55960/55960 - 37s - loss: 7.3149e-04 - mean_squared_error: 1.0213e-04 - va

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 56s - loss: 0.0821 - mean_squared_error: 0.0820 - val_loss: 0.0677 - val_mean_squared_error: 0.0673
Epoch 2/10
55960/55960 - 38s - loss: 0.0427 - mean_squared_error: 0.0420 - val_loss: 0.0681 - val_mean_squared_error: 0.0670
Epoch 3/10
55960/55960 - 38s - loss: 0.0024 - mean_squared_error: 0.0013 - val_loss: 0.0766 - val_mean_squared_error: 0.0757
Epoch 4/10
55960/55960 - 38s - loss: 0.0018 - mean_squared_error: 9.4124e-04 - val_loss: 0.0734 - val_mean_squared_error: 0.0726
Epoch 5/10
55960/55960 - 38s - loss: 9.5475e-04 - mean_squared_error: 2.2315e-04 - val_loss: 0.0763 - val_mean_squared_error: 0.0756
Epoch 6/10
55960/55960 - 38s - loss: 7.4515e-04 - mean_squared_error: 1.3508e-04 - va

Epoch 9/10
55960/55960 - 38s - loss: 5.3049e-04 - mean_squared_error: 1.3178e-04 - val_loss: 0.0765 - val_mean_squared_error: 0.0761
Epoch 10/10
55960/55960 - 38s - loss: 4.8244e-04 - mean_squared_error: 1.3248e-04 - val_loss: 0.0762 - val_mean_squared_error: 0.0759


{(1, 1): {'RMSE': 0.27},
 (2, 2): {'RMSE': 0.27},
 (4, 4): {'RMSE': 0.27},
 (8, 8): {'RMSE': 0.27},
 (29, 29): {'RMSE': 0.28},
 (128, 128): {'RMSE': 0.28},
 (256, 256): {'RMSE': 0.28}}

In [73]:
l2_reg_embedding_rmse = {}
for k in rand_parameters['l2_reg_embedding']:
    model = WDL(linear_feature_columns, dnn_feature_columns, dnn_hidden_units = (2,2), l2_reg_embedding = k, l2_reg_linear = 0.001, seed = 1024, task ='binary')
    model.compile("adam", "mse", metrics=['mse'])
    history = model.fit(train_data_input, train_data[target].values, batch_size=256, validation_split=0.2, epochs=10, verbose=2)
    pred = model.predict(test_data_input, batch_size=256)
    l2_reg_embedding_rmse[k]={"RMSE": np.round(math.sqrt(mean_squared_error(test_data[target].values, pred)),4)}
l2_reg_embedding_rmse

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 79s - loss: 0.1070 - mean_squared_error: 0.1047 - val_loss: 0.0779 - val_mean_squared_error: 0.0752
Epoch 2/10
55960/55960 - 46s - loss: 0.0787 - mean_squared_error: 0.0766 - val_loss: 0.0749 - val_mean_squared_error: 0.0732
Epoch 3/10
55960/55960 - 47s - loss: 0.0772 - mean_squared_error: 0.0757 - val_loss: 0.0741 - val_mean_squared_error: 0.0727
Epoch 4/10
55960/55960 - 39s - loss: 0.0766 - mean_squared_error: 0.0753 - val_loss: 0.0736 - val_mean_squared_error: 0.0723
Epoch 5/10
55960/55960 - 36s - loss: 0.0763 - mean_squared_error: 0.0750 - val_loss: 0.0733 - val_mean_squared_error: 0.0720
Epoch 6/10
55960/55960 - 40s - loss: 0.0760 - mean_squared_error: 0.0747 - val_loss: 0.0731 - val

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 62s - loss: 0.1064 - mean_squared_error: 0.1041 - val_loss: 0.0777 - val_mean_squared_error: 0.0751
Epoch 2/10
55960/55960 - 38s - loss: 0.0786 - mean_squared_error: 0.0766 - val_loss: 0.0749 - val_mean_squared_error: 0.0732
Epoch 3/10
55960/55960 - 37s - loss: 0.0772 - mean_squared_error: 0.0757 - val_loss: 0.0740 - val_mean_squared_error: 0.0727
Epoch 4/10
55960/55960 - 43s - loss: 0.0766 - mean_squared_error: 0.0753 - val_loss: 0.0736 - val_mean_squared_error: 0.0723
Epoch 5/10
55960/55960 - 42s - loss: 0.0763 - mean_squared_error: 0.0750 - val_loss: 0.0733 - val_mean_squared_error: 0.0720


Epoch 6/10
55960/55960 - 37s - loss: 0.0760 - mean_squared_error: 0.0747 - val_loss: 0.0731 - val_mean_squared_error: 0.0717
Epoch 7/10
55960/55960 - 38s - loss: 0.0758 - mean_squared_error: 0.0744 - val_loss: 0.0729 - val_mean_squared_error: 0.0715
Epoch 8/10
55960/55960 - 37s - loss: 0.0756 - mean_squared_error: 0.0742 - val_loss: 0.0728 - val_mean_squared_error: 0.0713
Epoch 9/10
55960/55960 - 37s - loss: 0.0755 - mean_squared_error: 0.0741 - val_loss: 0.0727 - val_mean_squared_error: 0.0712
Epoch 10/10
55960/55960 - 37s - loss: 0.0754 - mean_squared_error: 0.0739 - val_loss: 0.0726 - val_mean_squared_error: 0.0710


make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 87s - loss: 0.1068 - mean_squared_error: 0.1045 - val_loss: 0.0778 - val_mean_squared_error: 0.0751
Epoch 2/10
55960/55960 - 50s - loss: 0.0786 - mean_squared_error: 0.0766 - val_loss: 0.0749 - val_mean_squared_error: 0.0732
Epoch 3/10
55960/55960 - 50s - loss: 0.0772 - mean_squared_error: 0.0757 - val_loss: 0.0740 - val_mean_squared_error: 0.0727
Epoch 4/10
55960/55960 - 49s - loss: 0.0766 - mean_squared_error: 0.0753 - val_loss: 0.0736 - val_mean_squared_error: 0.0723
Epoch 5/10
55960/55960 - 31s - loss: 0.0763 - mean_squared_error: 0.0750 - val_loss: 0.0733 - val_mean_squared_error: 0.0720
Epoch 6/10
55960/55960 - 30s - loss: 0.0760 - mean_squared_error: 0.0747 - val_loss: 0.0731 - val

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'


Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 55s - loss: 0.1069 - mean_squared_error: 0.1041 - val_loss: 0.0769 - val_mean_squared_error: 0.0743
Epoch 2/10
55960/55960 - 33s - loss: 0.0780 - mean_squared_error: 0.0761 - val_loss: 0.0743 - val_mean_squared_error: 0.0729
Epoch 3/10
55960/55960 - 33s - loss: 0.0768 - mean_squared_error: 0.0755 - val_loss: 0.0737 - val_mean_squared_error: 0.0724
Epoch 4/10
55960/55960 - 33s - loss: 0.0763 - mean_squared_error: 0.0750 - val_loss: 0.0733 - val_mean_squared_error: 0.0719
Epoch 5/10
55960/55960 - 33s - loss: 0.0759 - mean_squared_error: 0.0745 - val_loss: 0.0729 - val_mean_squared_error: 0.0714
Epoch 6/10
55960/55960 - 33s - loss: 0.0757 - mean_squared_error: 0.0740 - val_loss: 0.0726 - val_mean_squared_error: 0.0710
Epoch 7/10
55960/55960 - 34s - loss: 0.0755 - mean_squared_error: 0.0737 - val_loss: 0.0726 - val_mean_squared_error: 0.0707
Epoch 8/10
55960/55960 - 34s - loss: 0.0754 - mean_squared_error: 0.0734 - 

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
Train on 55960 samples, validate on 13990 samples
Epoch 1/10
55960/55960 - 71s - loss: 0.1132 - mean_squared_error: 0.1041 - val_loss: 0.0777 - val_mean_squared_error: 0.0751
Epoch 2/10
55960/55960 - 38s - loss: 0.0787 - mean_squared_error: 0.0766 - val_loss: 0.0749 - val_mean_squared_error: 0.0732
Epoch 3/10
55960/55960 - 39s - loss: 0.0772 - mean_squared_error: 0.0757 - val_loss: 0.0741 - val_mean_squared_error: 0.0727
Epoch 4/10
55960/55960 - 38s - loss: 0.0766 - mean_squared_error: 0.0753 - val_loss: 0.0736 - val_mean_squared_error: 0.0723
Epoch 5/10
55960/55960 - 39s - loss: 0.0763 - mean_squared_error: 0.0750 - val_loss: 0.0733 - val_mean_squared_error: 0.0720
Epoch 6/10
55960/55960 - 39s - loss: 0.0760 - mean_squared_error: 0.0747 - val_loss: 0.0731 - val

{1e-07: {'RMSE': 0.271},
 1e-05: {'RMSE': 0.2709},
 0.001: {'RMSE': 0.271},
 0.1: {'RMSE': 0.2693},
 1: {'RMSE': 0.271}}

In [ ]:
l2_reg_linear_rmse = {}
for k in rand_parameters['l2_reg_linear']:
    model = WDL(linear_feature_columns, dnn_feature_columns, dnn_hidden_units = (2,2), l2_reg_linear = k, seed = 1024, task ='binary')
    model.compile("adam", "mse", metrics=['mse'])
    history = model.fit(train_data_input, train_data[target].values, batch_size=256, validation_split=0.2, epochs=10, verbose=2)
    pred = model.predict(test_data_input, batch_size=256)
    l2_reg_linear_rmse[k]={"RMSE": np.round(math.sqrt(mean_squared_error(test_data[target].values, pred)),2)}
l2_reg_embedding_rmse